
# EDA_Notebook.ipynb
This notebook contains step-by-step exploratory data analysis for the Medical Billing Claim Analysis project.
It loads `medical_claims.csv`, performs cleaning, shows key metrics, visualizations, and example model-ready feature engineering.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 50)
df = pd.read_csv('medical_claims.csv', parse_dates=['Paid_Date','Claim_Submitted_Date','Date_of_Service'])
df.head()

## 1. Basic data checks

In [ ]:
print('Rows, Columns:', df.shape)
print('\nMissing values by column:')
print(df.isna().sum())

print('\nData types:')
print(df.dtypes)

## 2. Denial rate and revenue loss

In [ ]:
denial_rate = (df[df['Claim_Status']=='Denied'].shape[0] / len(df)) * 100
revenue_loss = (df.loc[df['Claim_Status']=='Denied', 'Billed_Amount'] - df.loc[df['Claim_Status']=='Denied', 'Paid_Amount']).sum()
print(f'Denial rate: {denial_rate:.2f}%')
print(f'Estimated revenue loss from denied claims: ${revenue_loss:,.2f}')

## 3. Top denial reasons and insurance providers with most denials

In [ ]:
top_reasons = df[df['Claim_Status']=='Denied']['Denial_Reason'].value_counts().head(10)
print(top_reasons)

provider_denials = df[df['Claim_Status']=='Denied']['Insurance_Provider'].value_counts()
print(provider_denials)

## 4. SLA performance and average days by provider

In [ ]:
avg_sla = df.groupby('Insurance_Provider')['SLA_Days'].mean().sort_values(ascending=False)
print(avg_sla)

# Visualize denial rate by month
monthly = df.copy()
monthly['year_month'] = monthly['Date_of_Service'].dt.to_period('M')
monthly_group = monthly.groupby('year_month').apply(lambda x: (x['Claim_Status']=='Denied').sum()/len(x)).reset_index(name='denial_rate')
monthly_group['year_month'] = monthly_group['year_month'].astype(str)
monthly_group.plot(x='year_month', y='denial_rate', marker='o', figsize=(10,4))
plt.xticks(rotation=45)
plt.title('Monthly Denial Rate')
plt.tight_layout()
plt.show()

## 5. Feature engineering suggestions (for modeling denials)

In [ ]:
df['is_high_amount'] = (df['Billed_Amount']>1000).astype(int)
df['days_to_payment'] = (df['Paid_Date'] - df['Date_of_Service']).dt.days
# pending/denied will have NaN days_to_payment; fill with a large number for modeling
df['days_to_payment'] = df['days_to_payment'].fillna(999)
df[['Billed_Amount','is_high_amount','days_to_payment']].head()

## 6. Save a small sample for demo or upload to GitHub

In [ ]:
df.sample(10).to_csv('sample_claims_10.csv', index=False)
print('Saved sample_claims_10.csv')